# Übung 4.1 : Das Traglinienverfahren und die Prandtl'sche Tragflügeltheorie

<div style="text-align: justify"> </p>Die Übung 4 ist die erste von insgesamt fünf computer-basierten Übungen und soll einen ersten Einblick in aerodynamische Optimierungsaufgaben geben und wie sie mit Algorithmen gelöst werden können. </p>

Die **Prandtl'sche Traglinientheorie (engl. "lifting line theory")** besteht seit der Zeit des ersten Weltkrieges und findet heutzutage immernoch Anwendung. Darauf basierend wird das numerische Traglinienverfahren als analytische Methode eingesetzt, um im Flügel-Vorentwurf Vorhersagen über Effizienz und Verluste von Tragflügeln endlicher Streckung zu treffen.

#### Was liefert die Traglinientheorie?
- Auftriebsverteilung entlang der Spannweite
- Auftriebskraft 
- Induzierter Widerstand (Effizienz)

#### Was sind die Einschränkungen?
- gilt nur für große Streckungen
- Pfeilung kann nicht berücksichtigt werden 
- gilt nur für kleine Anstellwinkel

Die ausführliche Theorie zum Verfahren findet ihr in den Skripten begleitend zur Vorlesung. Nachfolgend soll nun exemplarisch gezeigt werden, wie ein solches Traglinienverfahren in Python implementiert werden kann
 </div>

In [272]:
# Importieren von Biblioteken
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, IntSlider, FloatSlider, fixed

## Aufgabe 4.1 a)
Untersuchen Sie am Beispiel des Rechteckflügels mit $l_i=1$ und $\Lambda = 10$ das Konvergenzverhalten der aerodynamischen Beiwerte ($C_W$, $C_A$) und des Oswald Faktors hinsichtlich der Abhängigkeit von der Anzahl der Elementarflügel (räumliche Diskretisierung).
Entscheiden Sie sich für eine Diskretisierung für alle folgenden Aufgabenteil und begründen Sie diese Entscheidung.

### 4.1.1 Visualisierung: Doppeltrapezflügel

In [273]:
# TODO elliptische Planform gleicher Streckung hinterlegen
def doppeltrapezfluegel(l_a, l_k, AR, l_i=1):
    b = AR * (l_k*0.75 + 0.5*(l_i-l_k)*0.75+l_a*(1-0.75)+0.5*(l_k-l_a)*0.25)
    s = b / 2
    y_lk = 0.75 * s 

    # Definition der Punkte für eine Seite
    points = np.array([
        [0, l_i/2],              
        [y_lk, l_k/2],    
        [s, l_a/2],         
        [s, -l_a/2],         
        [y_lk, -l_k/2],         
        [0, -l_i/2],
        [-y_lk, -l_k/2],  
        [-s, -l_a/2],    
        [-s, l_a/2] ,         
        [-y_lk, l_k/2],
        [0, l_i/2]
    ])
    
    return points


def plot_doppeltrapezfluegel(l_a, l_k, AR):
    points = doppeltrapezfluegel(l_a, l_k, AR)

    plt.figure(figsize=(10, 6))
    plt.plot(points[:,0], points[:, 1], '-', color="black")
    plt.xlabel('Spannweite [m]')
    plt.ylabel('Flügeltiefe [m]')
    plt.title('Doppeltrapezflügel')
    plt.axis('equal')
    plt.show()

interact(plot_doppeltrapezfluegel, 
         l_a=FloatSlider(value=0.5, min=0, max=1.0, step=0.1, description='l_a'), 
         l_k=FloatSlider(value=1.0, min=0, max=1.0, step=0.1, description='l_k'), 
         AR=FloatSlider(value=10, min=5, max=20, step=1, description=r'$\Lambda$'))

interactive(children=(FloatSlider(value=0.5, description='l_a', max=1.0), FloatSlider(value=1.0, description='…

<function __main__.plot_doppeltrapezfluegel(l_a, l_k, AR)>

### 4.1.2 Geometrische Parameter und Randbedinungen

Für die anschließenden Berechnung

In [274]:
# Konstant
alpha = 1           # Anstellwinel [rad]
U_inf = 10          # Anströmgeschwindigkeit [m/s]
rho = 1.225         # Dichte [kg/m^3]
l_i = 1             # Flügeltiefe Wurzel [m]

# Variabel
l_a = 0.5           # Flügeltiefe außen [m]
l_k = 1             # Flügeltiefe kink [m]
AR = 10             # Streckung/ Aspect Ratio

#### Berechnung weiterer geometrischer Parameter

In [275]:
b = AR * (l_k*0.75 + 0.5*(l_i-l_k)*0.75+l_a*(1-0.75)+0.5*(l_k-l_a)*0.25)    # Spannweite
s = b / 2                                                                   # Halbspannweite
y_lk = 0.75 * s                                                             # Kink-Position bei 75% der Halbspannweite
S = b**2 / AR                                                               # Flügelfläche

# Ausgabe
print("Spannweite:                  ", "%.2f" % b, "m")
print("Halbspannweite:              ", "%.2f" % s, "m")
print("Kink-Position:               ", "%.2f" % y_lk, "m")
print("Flügelfläche:                ", "%.2f" % S, "m^2")

Spannweite:                   9.38 m
Halbspannweite:               4.69 m
Kink-Position:                3.52 m
Flügelfläche:                 8.79 m^2


#### Vergleich mit elliptischer Planform

In [276]:
h_ell = l_i/2 * (np.sqrt(1 - (y_lk / s)))                                   # ???                   
S_ell = np.pi * l_i * 0.5 * s                                               # entsprechende Fläche
AR_ell = b**2 / S_ell                                                       # entsprechende Streckung

# Ausgabe
print("???:                         ", "%.2f" % h_ell, "m")
print("Flügelfläche:                ", "%.2f" % S_ell, "m^2")
print("Streckung:                   ", "%.2f" % AR_ell)

???:                          0.25 m
Flügelfläche:                 7.36 m^2
Streckung:                    11.94


### 4.1.3 Diskretisierung der Spannweite

In [277]:
N = 500                         # Anzahl der Elementarflügel
delta_b = b/N                   # Breite eines Elementarflügels
y = np.linspace(-s, s, N)

In [278]:
def plot_doppeltrapezfluegel(l_a, l_k, AR,N):
    points = doppeltrapezfluegel(l_a, l_k, AR)
    
    # Diskretisierung der Halbspannweite
    y_values = np.linspace(-s, s, N+1)
    
    # Tiefe des Flügels an verschiedenen y-Positionen berechnen
    def tiefe(y):
        y = abs(y)
        if y <= y_lk:
            return 1 + (l_k - 1) * (y / y_lk)
        else:
            return l_k + (l_a - l_k) * ((y - y_lk) / (s - y_lk))

    chord_values = [tiefe(y) for y in y_values]
    
    # Plotten des Flügels
    plt.figure(figsize=(10, 6))
    plt.plot(points[:,0], points[:, 1], '-', color="black")

    # Diskretisierte Punkte entlang der Spannweite
    for i in range(N):
        plt.plot([y_values[i], y_values[i+1]], [chord_values[i]/2, chord_values[i+1]/2], 'r--')
        plt.plot([y_values[i], y_values[i+1]], [-chord_values[i]/2, -chord_values[i+1]/2], 'r--')
        plt.plot([y_values[i], y_values[i]], [-chord_values[i]/2, chord_values[i]/2], 'g-')
    
    plt.xlabel('Spannweite [m]')
    plt.ylabel('Flügeltiefe [m]')
    plt.title('Diskretisierung Doppeltrapezflügel')
    plt.axis('equal')
    plt.grid(True)
    plt.show()

# Gegebene Parameter
l_a = 0.5
l_k = 1.0
AR = 10

interact(plot_doppeltrapezfluegel, 
         l_a=fixed(l_a), 
         l_k=fixed(l_k), 
         AR=fixed(AR), 
         N=IntSlider(value=30, min=1, max=100, step=1, description='N'))

interactive(children=(IntSlider(value=30, description='N', min=1), Output()), _dom_classes=('widget-interact',…

<function __main__.plot_doppeltrapezfluegel(l_a, l_k, AR, N)>

### 4.1.4 Berechnung der Aufpunkt-Koordinaten

Zur Bestimmung der $N$ (Variable n) unbekannten Wirbelstärken $\Gamma$ wird die kinematische Strömungsbedingung in den $N$ (Variable $m$) Aufpunkten erfüllt, und man hat ein lineares Gleichungssystem zu lösen. 

Dafür müssen die Koordinaten der Aufpunkte bestimmt werden.

In [279]:
# Spannweitige Positionen der Punkte A, B und C
# TODO y_ai, y_bi und y_ci müssen mit laufvariablen für die N punkte bestimmt werden np.linspace
# darauf basierend werden dann die zugehörigen x Koordinaten berechnet
ya = y[:-1]             
yb = y[1:]
yc = (y[:-1] + y[1:]) / 2

# Tiefenposition der Punkte A, B und C
def x_position(y, l_i, l_k, l_a, y_lk):
    if y <= y_lk:
        return l_i / 4 * (1 - (y / y_lk)) + l_k / 4 * (y / y_lk)
    else:
        return l_k / 4 * (1 - ((y - y_lk) / (s - y_lk))) + l_a / 4 * ((y - y_lk) / (s - y_lk))

xa = np.array([x_position(y_i, l_i, l_k, l_a, y_lk) for y_i in ya])
xb = np.array([x_position(y_i, l_i, l_k, l_a, y_lk) for y_i in yb])
xc = np.array([x_position(y_i, l_i, l_k, l_a, y_lk) for y_i in yc])
